In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import py_code.input_data as input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#下载下来的数据集被分三个子集：
#5.5W行的训练数据集（mnist.train），
#5千行的验证数据集（mnist.validation)
#1W行的测试数据集（mnist.test）。
#因为每张图片为28x28的黑白图片，所以每行为784维的向量。
trainimg   = mnist.train.images                               
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels

In [5]:
print (trainimg.shape)
print (trainlabel.shape)
print (testimg.shape)
print (testlabel.shape)
print(mnist.train.next_batch)
#print (trainimg)
print (trainlabel[0])

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)
<bound method DataSet.next_batch of <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000022229C15CF8>>
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [6]:
x = tf.placeholder(shape=(None, 784), dtype=tf.float32)
y = tf.placeholder(shape=(None, 10), dtype=tf.float32)

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

#回归模型  w*x+b
actv=tf.nn.softmax(tf.matmul(x,W)+b)

cost=tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv),reduction_indices=1))

learning_rate=0.01

optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [7]:
training_epochs=50
#批尺寸
batch_size=100
# 训练迭代次数
display_step=5

In [8]:
pred=tf.equal(tf.argmax(actv,1),tf.argmax(y,1))
#正确率
accr=tf.reduce_mean(tf.cast(pred,"float"))
#初始化
init=tf.global_variables_initializer()

In [9]:
sess=tf.Session()
sess.run(init)

for epoch in range(training_epochs):
    avg_cost=0.
    #55000/100
    num_batch=int(mnist.train.num_examples/batch_size)
    for i in range(num_batch):
        #获取数据集 next_batch获取下一批的数据
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        #模型训练
        sess.run(optm,feed_dict={x:batch_xs,y:batch_ys})
        feeds={x:batch_xs,y:batch_ys}
        avg_cost+=sess.run(cost,feed_dict=feeds)/num_batch
    #满足5次的一个迭代
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        feeds_test = {x: mnist.test.images, y: mnist.test.labels}
        train_acc = sess.run(accr, feed_dict=feeds_train)
        test_acc = sess.run(accr, feed_dict=feeds_test)
        print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
               % (epoch, training_epochs, avg_cost, train_acc, test_acc))
print ("DONE")

Epoch: 000/050 cost: 1.176283828 train_acc: 0.840 test_acc: 0.853
Epoch: 005/050 cost: 0.440940045 train_acc: 0.860 test_acc: 0.895
Epoch: 010/050 cost: 0.383335689 train_acc: 0.920 test_acc: 0.905
Epoch: 015/050 cost: 0.357283105 train_acc: 0.910 test_acc: 0.908
Epoch: 020/050 cost: 0.341510462 train_acc: 0.930 test_acc: 0.912
Epoch: 025/050 cost: 0.330527925 train_acc: 0.870 test_acc: 0.914
Epoch: 030/050 cost: 0.322377158 train_acc: 0.900 test_acc: 0.916
Epoch: 035/050 cost: 0.315936385 train_acc: 0.900 test_acc: 0.917
Epoch: 040/050 cost: 0.310713485 train_acc: 0.940 test_acc: 0.918
Epoch: 045/050 cost: 0.306388705 train_acc: 0.930 test_acc: 0.919
DONE
